In [1]:
%matplotlib inline

In [2]:
import torch
import os
from torch.utils.data import Dataset
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torch import nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import finalprojectneuralnetworks as fpnn
import csv
import time

In [3]:
n = 5  # number of samples/batch size
l = 5  # sentence len
d = 3  # embedding dimension
rand_arr = torch.rand(n, l, d)

In [4]:
layer = nn.Conv1d(1, 2, 3, stride=1, padding=0)
print("layer: ", layer)
input = torch.randn(1, 140)
print("Input: ", input.size())
input = input.unsqueeze(0)
print("Input unsqueezed: ", input.size())
print(input)
output = layer(input)
print("output.size: ", output.size())
print("output: ", output)

layer:  Conv1d(1, 2, kernel_size=(3,), stride=(1,))
Input:  torch.Size([1, 140])
Input unsqueezed:  torch.Size([1, 1, 140])
tensor([[[-1.0380,  0.9593, -1.7358,  0.3515, -0.3954, -1.0867,  0.2999,
           0.7437, -0.8898, -1.4626,  0.2972, -0.3400,  0.0892, -0.7238,
           1.5188, -0.5569,  0.3697,  0.1859, -0.3181,  0.0132,  0.7181,
           0.5131,  0.3937, -1.6648,  1.5227,  0.6561, -0.1138,  0.9435,
           0.2878, -0.3212, -1.8038, -0.0031,  0.4088,  0.4923,  0.4005,
           0.7742,  1.1200,  1.2780, -0.9576,  0.2507, -1.2729, -1.0266,
           1.3719, -0.2354,  1.4240, -0.3867, -0.2262, -0.1108,  0.3872,
           0.5279, -1.1984, -0.1101, -0.7574,  0.0913,  0.3945, -0.0420,
           1.2125,  0.9220,  0.7476,  1.5025,  1.1631,  0.3153, -1.1844,
           1.2655,  1.2039, -0.4145,  1.4334,  0.2362,  1.4383,  0.7265,
          -1.3268, -1.2090,  1.0142,  0.1164, -1.9747,  0.0876, -3.0293,
           0.3912,  0.2769, -1.3204,  0.2490,  0.0884,  0.7548,  0.6329,


In [5]:
root_directory = "D:\College\CS_545\Final_Project"
# root_directory = "/s/chopin/l/grad/acf003/CS_545"

train_file = "EEG_Eye_State_Lumped_10_Normalized_Train.csv"
validate_file = "EEG_Eye_State_Lumped_10_Normalized_Validate.csv"
test_file = "EEG_Eye_State_Lumped_10_Normalized_Test.csv"

train_dataset = fpnn.FinalProjectEEGDataset(train_file, root_directory)
validate_dataset = fpnn.FinalProjectEEGDataset(validate_file, root_directory)
test_dataset = fpnn.FinalProjectEEGDataset(test_file, root_directory)
train_dataloader = DataLoader(train_dataset, batch_size=64)
validate_dataloader = DataLoader(validate_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

In [6]:
import torch.nn as nn
import torch.nn.functional as F

num_inputs = 140
num_channels = 1
kernel_size = 3
num_outputs = 2
num_outputs2 = 5


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(num_channels, num_outputs, kernel_size)
        self.conv2 = nn.Conv1d(num_outputs, num_outputs2, kernel_size)
        print((kernel_size - 1) * 2)
        self.fc1 = nn.Linear(
            (num_inputs - (kernel_size - 1) * 2) * num_outputs2, num_outputs
        )
        self.fc2 = nn.Linear(num_outputs, num_outputs)

    def forward(self, x):
        # print("x.shape1", x.shape)
        x = nn.functional.relu(self.conv1(x))
        # print("x.shape2", x.shape)

        x = nn.functional.relu(self.conv2(x))
        # print("x.shape3", x.shape)

        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        # print("x.shape4", x.shape)

        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x


model = Net().float()

4


In [7]:
print(model)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

Net(
  (conv1): Conv1d(1, 2, kernel_size=(3,), stride=(1,))
  (conv2): Conv1d(2, 5, kernel_size=(3,), stride=(1,))
  (fc1): Linear(in_features=680, out_features=2, bias=True)
  (fc2): Linear(in_features=2, out_features=2, bias=True)
)


In [8]:
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    model.train()
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs.float())
        loss = loss_fn(outputs, labels.long())
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:  # print every 2000 mini-batches
            print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print("Finished Training")

Finished Training


In [9]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in validate_dataloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = model(images.float())
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print("Accuracy of the network on the test inputs: %d %%" % (100 * correct / total))

Accuracy of the network on the test inputs: 49 %
